# Market Simulator
ML for trading Udacity Course exercise

More info:
http://wiki.quantsoftware.org/index.php?title=CompInvesti_Homework_3

A transcription of the Udacity Course lectures can be find on https://docs.google.com/document/d/1ELqlnuTSdc9-MDHOkV0uvSY4RmI1eslyQlU9DgOY_jc/edit?usp=sharing

Kairoart 2018



## Overview

In this project you will create a basic market simulator that accepts trading orders and keeps track of a portfolio's value and saves it to a file. You will also create another program that assesses the performance of that portfolio. 

## Part 1: Create a market simulation tool

Starting cash: 1000000  
Input file: orders.csv 
Output file: values.csv

The file of orders is organized like this:

    Year
    Month
    Day
    Symbol
    BUY or SELL
    Number of Shares 
    
For example:

    2008, 12, 3, AAPL, BUY, 130
    2008, 12, 8, AAPL, SELL, 130
    2008, 12, 5, IBM, BUY, 50


### Goal

Your simulator should calculate the total value of the portfolio for each day using adjusted closing prices (cash plus value of equities) and print the result to the file values.csv. The contents of the values.csv file should look something like this:

    2008, 12, 3, 1000000
    2008, 12, 4, 1000010
    2008, 12, 5, 1000250
    ...


### Import libraries

In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import operator
import sys
import csv
import math

# To fetch data
from pandas_datareader import data as pdr   
import fix_yahoo_finance as yf  
yf.pdr_override()   

# Add plotly for interactive charts
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools



### Read orders from file

In [2]:
def readOrdersFileIntoDF(filename):

    # opening the filename
    fr = open(filename)
    
    # for row count in 
    index=0
    
    # Lists used for making the dataframe.
    dtList = []
    symbolList = []
    orderTypeList = []
    volumeList = []
    
    # For each line
    # A Sample Line - 2011,1,14,AAPL,Buy,1500
    for orderString in fr.readlines():

        # Stripping off the return line character
        orderString=orderString.strip()
        
        # Splitting the line and getting a List back
        listFromLine = orderString.split(',')
        
        # Adding the dates into dtList. 16,00,00 for 1600 hrs
        dtList.append(dt.datetime(int(listFromLine[0]), int(listFromLine[1]), int(listFromLine[2])))
        
        # Adding the symbols into symbolList
        symbolList.append(listFromLine[3])
        
        # Adding the orders into orderTypeList
        orderTypeList.append(listFromLine[4])
        
        # Adding the number of shares into volumeList
        volumeList.append(listFromLine[5])

    # Creating a Dictionary for converting it into DataFrame later
    data = { 'datetime' : dtList, 'symbol' : symbolList, 'ordertype':orderTypeList, 'volume':volumeList }

    # Converting the Dictinary into a nice looking Pandas Dataframe ordered by datetime index
    ordersDataFrame = pd.DataFrame(data)
    ordersDataFrame.index= ordersDataFrame['datetime']
    
    # Drop datetime column
    ordersDataFrame.drop('datetime', axis=1, inplace=True)
    #print(ordersDataFrame)
    
    
    # Getting the Symbols from the Orders. This list will be required for fetching the prices
    symbolList = list(set(ordersDataFrame['symbol']))
    
    # Returning it.
    return ordersDataFrame, symbolList

### Get data from Yahoo for the given dates 

In [12]:
def fetchData(dt_start, dt_end, ls_symbols):

    # Add a day to dt_end for Yahoo purpose
    dt_end = pd.to_datetime(dt_end) + pd.DateOffset(days=1)
    
    # Get data of trading days between the start and the end.
    df = pdr.get_data_yahoo(
            # tickers list (single tickers accepts a string as well)
            tickers = ls_symbols,

            # start date (YYYY-MM-DD / datetime.datetime object)
            # (optional, defaults is 1950-01-01)
            start = dt_start,

            # end date (YYYY-MM-DD / datetime.datetime object)
            # (optional, defaults is Today)
            end = dt_end,

            # return a multi-index dataframe
            # (optional, default is Panel, which is deprecated)
            as_panel = False,

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            group_by = 'ticker',

            # adjust all OHLC automatically
            # (optional, default is False)
            auto_adjust = False
    )
        

    # Getting the numpy ndarray of Adj Close prices.
    adj_close_price = df.loc[:, (slice(None), ('Adj Close'))]

    
    # returning the Adj Closed prices for all the days    
    return adj_close_price

### Market simulator

In [13]:
def marketsim(initialCash, ordersdf, symbols):

    # reading the boundary dates
    dt_start = ordersdf.index[0]
    dt_end = ordersdf.index[len(ordersdf)-1]
    
    # All the adjustedClosingPrices fetched from NYSE within the range and for given symbols
    closingPrices = fetchData(dt_start, dt_end, symbols)
    num_tradingDays = len(closingPrices)
    print(closingPrices)
    
    #Cash for the days
    temp = np.zeros((num_tradingDays, 1))
    cash = pd.DataFrame(temp, columns = ['cashinhand'])
    
    #Value for the days
    temp = np.zeros((num_tradingDays, 1))
    valueFrame = pd.DataFrame(temp, columns = ['valueOfPortfolio'])

    #Setting the first value to be the initial cash amount.
    cash.cashinhand.loc[0] = initialCash
    
    index = 0
    
    for tradingDayIndex in range(num_tradingDays):  
        if tradingDayIndex != 0:
            cash.cashinhand.iloc[tradingDayIndex] = cash.cashinhand.iloc[tradingDayIndex - 1] 
        else:
            cash.cashinhand.iloc[tradingDayIndex] = initialCash
            
        for tradingOrder in ordersDataFrame.index:
            if tradingOrder == closingPrices.index[tradingDayIndex]:
                if ordersdf.ordertype.iloc[index] == 'Buy':
                    toBuySymbol = ordersdf.symbol.iloc[index]
                    toBuy = symbols.index(toBuySymbol)
                    numShares = ordersdf.volume.iloc[index]
                    priceForTheDay = closingPrices.iloc[tradingDayIndex][toBuy]
                    cash.cashinhand.iloc[tradingDayIndex] = cash.cashinhand.iloc[tradingDayIndex] - (priceForTheDay * float(numShares))
                    print("Date: ", tradingOrder,
                          "Symbol: ", toBuySymbol,
                          "Order: ", ordersdf.ordertype.iloc[index],
                          "Shares: ", numShares,
                          "Price: ", priceForTheDay,
                          "Cash: ", cash.cashinhand.iloc[tradingDayIndex] )
                elif ordersdf.ordertype.iloc[index] == 'Sell':
                    toSellSymbol = ordersdf.symbol.iloc[index]
                    toSell = symbols.index(toSellSymbol)
                    numShares = ordersdf.volume.iloc[index]
                    priceForTheDay = closingPrices.iloc[tradingDayIndex][toSell]
                    cash.cashinhand.iloc[tradingDayIndex] = cash.cashinhand.iloc[tradingDayIndex] + (priceForTheDay * float(numShares))
                    print("Date: ", tradingOrder,
                          "Symbol: ", toSellSymbol,
                          "Order: ", ordersdf.ordertype.iloc[index],
                          "Shares: ", numShares,
                          "Price: ", priceForTheDay,
                          "Cash: ", cash.cashinhand.iloc[tradingDayIndex] )
                else:
                    print("error")
                index+=1
        

        valueFrame['valueOfPortfolio'] = cash['cashinhand'].values
        
    valueFrame.index = closingPrices.index

    return valueFrame

### Write values to file

In [14]:
def writeValuesIntoCSV(valuesFilename, valueFrame):
    file = open(valuesFilename, 'w')
    writer = csv.writer(file)
    for index in range(len(valueFrame)):
        writer.writerow([valueFrame.index[index].year, valueFrame.index[index].month, valueFrame.index[index].day ,int(round(valueFrame.valueOfPortfolio.iloc[index], 0))])
    
    file.close()

### Define initial values and execute simulation

In [15]:
initialCash = 1000000
ordersFilename = 'input/orders.csv'
valuesFilename = 'output/values.csv'

# Reading the data from the file, and getting a NumPy matrix
ordersDataFrame, symbols = readOrdersFileIntoDF(ordersFilename)

# Getting data from marketsim
print("List of transactions")
valueFrame = marketsim(initialCash, ordersDataFrame, symbols)

# Writing data to file
writeValuesIntoCSV(valuesFilename, valueFrame)

List of transactions
[*********************100%***********************]  4 of 4 downloaded
                  XOM                                                        \
                 Open       High        Low      Close  Adj Close    Volume   
Date                                                                          
2011-01-10  75.129997  75.269997  74.580002  75.129997  59.117691  20766400   
2011-01-11  75.019997  75.860001  75.019997  75.690002  59.558342  15805400   
2011-01-12  75.940002  76.620003  75.669998  76.580002  60.258648  17363000   
2011-01-13  76.650002  76.820000  75.980003  76.709999  60.360943  20438100   
2011-01-14  76.669998  78.000000  76.500000  77.839996  61.250114  23478700   
2011-01-18  77.790001  78.849998  77.720001  78.709999  61.934692  23883600   
2011-01-19  78.629997  78.930000  78.129997  78.239998  61.564857  21844100   
2011-01-20  77.980003  78.160004  76.769997  77.750000  61.179298  29121600   
2011-01-21  78.209999  79.099998  78.059

## Part 2: Create a portfolio analysis tool

The tool should read in the daily values (cumulative portfolio value) from values.csv and plot them. It should use the symbol on the command line as a benchmark for comparison (in this case $SPX). Using this information, it should:

    * Plot the price history over the trading period.
    * Your program should also output:
        * Standard deviation of daily returns of the total portfolio
        * Average daily return of the total portfolio
        * Sharpe ratio (Always assume you have 252 trading days in an year. And risk free rate = 0) of the total portfolio
        * Cumulative return of the total portfolio 


### Read cumulative portfolio values

In [ ]:
def readValuesFileIntoDF(filename):

    # opening the filename
    fr = open(filename)
    
    # for row count in 
    index=0
    
    # Lists used for making the dataframe.
    # Date list    
    dtList = []
    
    # Cumulative values list
    cvList = []
    
    # For each line
    # A Sample Line - 2011,1,14,AAPL,Buy,1500
    for orderString in fr.readlines():

        # Stripping off the return line character
        orderString=orderString.strip()
        
        # Splitting the line and getting a List back
        listFromLine = orderString.split(',')
        
        # Adding the dates into dtList.
        dtList.append(dt.datetime(int(listFromLine[0]), int(listFromLine[1]), int(listFromLine[2])))
                
        # Adding the number of shares into volumeList
        cvList.append(listFromLine[3])

    # Creating a Dictionary for converting it into DataFrame later
    data = { 'datetime' : dtList, 'Portfolio Value':cvList }

    # Converting the Dictinary into a nice looking Pandas Dataframe ordered by datetime index
    valuesDataFrame = pd.DataFrame(data)
    valuesDataFrame.index= valuesDataFrame['datetime']
    
    # Drop datetime column
    valuesDataFrame.drop('datetime', axis=1, inplace=True)
    #print(ordersDataFrame)

    
    # Returning it.
    return valuesDataFrame

In [ ]:
# Funds values
valuesFilename = 'output/values.csv'
df = readValuesFileIntoDF(valuesFilename)
#print(df.head())


### Fecth SPX data for comparison

In [ ]:
# reading the boundary dates
dt_start = df.index[0]
dt_end = df.index[len(df)-1]

symbolSPX = ['SPX']
spxValues = pdr.get_data_yahoo(
            # tickers list (single tickers accepts a string as well)
            tickers = symbolSPX,

            # start date (YYYY-MM-DD / datetime.datetime object)
            # (optional, defaults is 1950-01-01)
            start = dt_start,

            # end date (YYYY-MM-DD / datetime.datetime object)
            # (optional, defaults is Today)
            end = dt_end,

            # return a multi-index dataframe
            # (optional, default is Panel, which is deprecated)
            as_panel = False,

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            group_by = 'ticker',

            # adjust all OHLC automatically
            # (optional, default is False)
            auto_adjust = False
    )


# Convert string to number
spxValues['Adj Close'] = pd.to_numeric(spxValues['Adj Close'], errors='coerce')
portf_SPX_value = pd.Series(spxValues['Adj Close'])
#print(portf_SPX_value)


### Plot portfolio value from values.csv

In [ ]:


trace_value = go.Scatter(
                x=df.index,
                y=df['Portfolio Value'],
                name = "Portfolio Value",
                line = dict(color = '#17BECF'),
                opacity = 0.8)
trace_dr = go.Scatter(
                x=spxValues.index,
                y=spxValues['Adj Close'],
                name = "SPX",
                line = dict(color = '#FF8000'),
                opacity = 0.8)
data = [trace_value]

layout = dict(
    title = "Cumulative Portfolio Value",
    xaxis = dict(
            title='Dates',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(step='all')
                ])
            ),
            ),

    yaxis = dict(
            title='Value')

    )



fig = dict(data=data, layout=layout)
iplot(fig)

### Plot the price history over the trading period

In [ ]:
# reading the boundary dates
dt_start = df.index[0]
dt_end = df.index[len(df)-1]

# Getting data from Yahoo
values = fetchData(dt_start, dt_end, symbols)

# Getting the symbol names
columnNames = list(values.head(1)) 
symbols_dict = dict(columnNames)
columnNames = list(symbols_dict.keys()) 

In [ ]:

trace_price1 = go.Scatter(
                x=df.index,
                y=values.iloc[:,0],
                name = columnNames[0],
                line = dict(color = '#17BECF'),
                opacity = 0.8)
trace_price2 = go.Scatter(
                x=df.index,
                y=values.iloc[:,1],
                name = columnNames[1],
                line = dict(color = '#FF8000'),
                opacity = 0.8)
trace_price3 = go.Scatter(
                x=df.index,
                y=values.iloc[:,2],
                name = columnNames[2],
                line = dict(color = '#FF0000'),
                opacity = 0.8)
trace_price4 = go.Scatter(
                x=df.index,
                y=values.iloc[:,3],
                name = columnNames[3],
                line = dict(color = '#04B404'),
                opacity = 0.8)
data = [trace_price1, trace_price2, trace_price3, trace_price4]

layout = dict(
    title = "Price History over the trading period",
    xaxis = dict(
            title='Dates',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(step='all')
                ])
            ),
            ),

    yaxis = dict(
            title='Price')

    )



fig = dict(data=data, layout=layout)
iplot(fig)

### Standard deviation of daily returns of the total portfolio 

In [ ]:
# Get daily portfolio value
valuesFilename = 'output/values.csv'
df = readValuesFileIntoDF(valuesFilename)

# Convert string to number
df['Portfolio Value'] = pd.to_numeric(df['Portfolio Value'], errors='coerce')
portf_value = pd.Series(df['Portfolio Value'])


# Daily returns funds
daily_rets = portf_value.pct_change(1)
daily_rets = daily_rets[1:]  

# daily returns normedSPX
daily_rets_SPX = portf_SPX_value.pct_change()
daily_rets_SPX = daily_rets_SPX[1:]

trace_daily_rets = go.Scatter(
                x=df.index,
                y=daily_rets,
                name = "Funds",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

trace_daily_rets_SPX = go.Scatter(
                x=df.index,
                y=daily_rets_SPX,
                name = "SPX",
                line = dict(color = '#FF8000'),
                opacity = 0.8)

data = [trace_daily_rets, trace_daily_rets_SPX]

layout = dict(
    title = "Daily Returns of the total Portfolio",
    xaxis = dict(
            title='Dates',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(step='all')
                ])
            ),
            ),

    yaxis = dict(
            title='')

    )



fig = dict(data=data, layout=layout)
iplot(fig)

# Standar deviation daily return
stddevPort = daily_rets.std()

# Standar deviation SPX daily return
stddevSPX = daily_rets_SPX.std()



### Average daily return of the total portfolio 

In [ ]:
# Averrage daily returns
averagePortfolioDailyRets = daily_rets.mean()

# Averrage SPX daily returns
averageSPXDailyRets = daily_rets_SPX.mean()

### Sharpe ratio 

(Always assume you have 252 trading days in an year. And risk free rate = 0) of the total portfolio 

In [ ]:
# Sharpe ratio funds
sharpeRatioPort = averagePortfolioDailyRets / stddevPort
k = math.sqrt(252)
sharpeRatioPort = k * sharpeRatioPort

# Sharpe ratio SPX
sharpeRatioSPX = averageSPXDailyRets / stddevSPX
k = math.sqrt(252)
sharpeRatioSPX = k * sharpeRatioSPX

### Cumulative return of the total portfolio

In [ ]:
# Cumulative return funds
totalPortRet = (portf_value[-1] / portf_value[0] -1)

print(portf_value[-1])
print(portf_value[0] -1)

# Cumulative return SPX
totalSPXRet = (portf_SPX_value[-1] / portf_SPX_value[0] -1)

## Results

In [ ]:
print("The final value of the portfolio using the sample file is ", df.index[-1], " ", df['Portfolio Value'].iloc[-1])
print("Details of the Performance of the portfolio")
print("")

print("Data Range :", dt_start ," to ", dt_end)

print("")

print("Sharpe Ratio of Fund :", sharpeRatioPort)
print("Sharpe Ratio of $SPX :", sharpeRatioSPX)
print("")

print("Total Return of Fund :", totalPortRet)
print("Total Return of $SPX :", totalSPXRet)

print("")

print("Standard Deviation of Fund :", stddevPort)
print("Standard Deviation of $SPX :", stddevSPX)

print("")

print("Average Daily Return of Fund :", averagePortfolioDailyRets)
print("Average Daily Return of $SPX :", averageSPXDailyRets)

## Conclusions

In this example with the data got from orders.csv file we can conclude the following:

1. **Sharpe ratio:** The risk free rate of return is higher in our funds (0.89) than in SPX, 
2. **Total return or cumulative return:** We'll obtain a 15.20% over our initial investment.
3. **Standard Deviation or Volatility:** In this case SPX is more volatile than our fund.
4. **Average Daily Return:** We've made a 1.1% with our funds, almost the same as SPX